<a href="https://colab.research.google.com/github/root2116/direct-feedback-alignment/blob/main/dfa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 追実験1

In [5]:
import cupy as np
import torch
from torchvision import datasets, transforms
%matplotlib inline
import matplotlib.pyplot as plt

from sklearn.manifold import TSNE


def softmax(x):
    # ベクトルxの各要素から最大値を引く（オーバーフロー防止のため）
    e_x = np.exp(x - np.max(x))
    # 要素ごとのexpの和で各要素を割り、確率として解釈できるようにする
    return e_x / e_x.sum(axis=0)

def sigmoid(x):
    return 1 / (1+ np.exp(-x))

def initialize_bp_params():
    np.random.seed(1)
    W1 = np.random.randn(400, 784) * 0.01
    b1 = np.zeros((400, 1))
    W2 = np.random.randn(400, 400) * 0.01
    b2 = np.zeros((400, 1))
    W3 = np.random.randn(400, 400) * 0.01
    b3 = np.zeros((400, 1))
    W4 = np.random.randn(10, 400) * 0.01
    b4 = np.zeros((10, 1))

    params = {  "W1": W1,
                "b1": b1,
                "W2": W2,
                "b2": b2,
                "W3": W3,
                "b3": b3,
                "W4": W4,
                "b4": b4}

    return params

def initialize_dfa_params():
    np.random.seed(1)
    W1 = np.zeros((400, 784))
    b1 = np.zeros((400, 1))
    W2 = np.zeros((400, 400))
    b2 = np.zeros((400, 1))
    W3 = np.zeros((400, 400))
    b3 = np.zeros((400, 1))
    W4 = np.zeros((10, 400))
    b4 = np.zeros((10, 1))

    B1 = np.random.randn(400, 10) * 0.01
    B2 = np.random.randn(400, 10) * 0.01
    B3 = np.random.randn(400, 10) * 0.01

    params = {  "W1": W1,
                "b1": b1,
                "W2": W2,
                "b2": b2,
                "W3": W3,
                "b3": b3,
                "W4": W4,
                "b4": b4,
                "B1": B1,
                "B2": B2,
                "B3": B3}

    return params

def forward_propagation(X, params):
    W1 = params["W1"]
    b1 = params["b1"]
    W2 = params["W2"]
    b2 = params["b2"]
    W3 = params["W3"]
    b3 = params["b3"]
    W4 = params["W4"]
    b4 = params["b4"]

    A1 = np.dot(W1, X) + b1
    H1 = np.tanh(A1)
    A2 = np.dot(W2, H1) + b2
    H2 = np.tanh(A2)
    A3 = np.dot(W3, H2) + b3
    H3 = np.tanh(A3)
    AY = np.dot(W4, H3) + b4
    # Y_ = softmax(AY)
    Y_ = sigmoid(AY)

    cache = {"A1": A1,
            "H1": H1,
            "A2": A2,
            "H2": H2,
            "A3": A3,
            "H3": H3,
            "AY": AY,
            "Y_": Y_}

    return Y_, cache


def backward_propagation(params, cache, X, Y):
    # batch size
    N = X.shape[1]

    W1 = params["W1"]
    W2 = params["W2"]
    W3 = params["W3"]
    W4 = params["W4"]

    H1 = cache["H1"]
    H2 = cache["H2"]
    H3 = cache["H3"]
    Y_ = cache["Y_"]

    dY_ = (Y_ - Y)
    dW4 = (1 / N) * np.dot(dY_, H3.T)
    db4 = (1 / N) * np.sum(dY_, axis=1, keepdims=True)

    dH3 = np.dot(W4.T, dY_)
    dA3 = dH3 * (1 - np.power(H3, 2))
    dW3 = (1 / N) * np.dot(dA3, H2.T)
    db3 = (1 / N) * np.sum(dA3, axis=1, keepdims=True)

    dH2 = np.dot(W3.T, dA3)
    dA2 = dH2 * (1 - np.power(H2, 2))
    dW2 = (1 / N) * np.dot(dA2, H1.T)
    db2 = (1 / N) * np.sum(dA2, axis=1, keepdims=True)

    dH1 = np.dot(W2.T, dA2)
    dA1 = dH1 * (1 - np.power(H1, 2))
    dW1 = (1 / N) * np.dot(dA1, X.T)
    db1 = (1 / N) * np.sum(dA1, axis=1, keepdims=True)

    gradients = {"dW1": dW1,
                "db1": db1,
                "dW2": dW2,
                "db2": db2,
                "dW3": dW3,
                "db3": db3,
                "dW4": dW4,
                "db4": db4}

    return gradients

def direct_feedback_alignment(params, cache, X, Y):
    # batch size
    N = X.shape[1]

    B1 = params["B1"]
    B2 = params["B2"]
    B3 = params["B3"]

    H1 = cache["H1"]
    H2 = cache["H2"]
    H3 = cache["H3"]
    Y_ = cache["Y_"]

    dY_ = Y_ - Y
    dW4 = (1 / N) * np.dot(dY_, H3.T)
    db4 = (1 / N) * np.sum(dY_, axis=1, keepdims=True)

    dH3 = np.dot(B3, dY_)
    dA3 = dH3 * (1 - np.power(H3, 2))
    dW3 = (1 / N) * np.dot(dA3, H2.T)
    db3 = (1 / N) * np.sum(dA3, axis=1, keepdims=True)

    dH2 = np.dot(B2, dY_)
    dA2 = dH2 * (1 - np.power(H2, 2))
    dW2 = (1 / N) * np.dot(dA2, H1.T)
    db2 = (1 / N) * np.sum(dA2, axis=1, keepdims=True)

    dH1 = np.dot(B1, dY_)
    dA1 = dH1 * (1 - np.power(H1, 2))
    dW1 = (1 / N) * np.dot(dA1, X.T)
    db1 = (1 / N) * np.sum(dA1, axis=1, keepdims=True)

    gradients = {"dW1": dW1,
                "db1": db1,
                "dW2": dW2,
                "db2": db2,
                "dW3": dW3,
                "db3": db3,
                "dW4": dW4,
                "db4": db4}

    return gradients

def update_params(params, grads, learning_rate=0.01):
    params["W1"] -= learning_rate * grads["dW1"]
    params["b1"] -= learning_rate * grads["db1"]
    params["W2"] -= learning_rate * grads["dW2"]
    params["b2"] -= learning_rate * grads["db2"]
    params["W3"] -= learning_rate * grads["dW3"]
    params["b3"] -= learning_rate * grads["db3"]
    params["W4"] -= learning_rate * grads["dW4"]
    params["b4"] -= learning_rate * grads["db4"]
    return params


def compute_loss(Y_, Y):

    N = Y.shape[1]
    # loss = (-1 / N) * np.sum(np.multiply(Y, np.log(Y_)))
    loss = (-1 / N) * np.sum(np.multiply(Y, np.log(Y_)) + np.multiply(1 - Y, np.log(1 - Y_)))
    loss = np.squeeze(loss)

    return loss


def train(backward_propagation, params, train_loader, test_loader, learning_rate=0.01, num_epochs=100):
    train_error_list = []
    test_error_list = []

    for epoch in range(num_epochs):
        total_loss = 0
        total_correct = 0
        total_samples = 0
        for i, (images, labels) in enumerate(train_loader):
            X = images.view(-1, 28 * 28)
            X = np.asarray(X).T
            Y = np.eye(10)[labels]
            Y = np.asarray(Y).T

            labels_gpu = np.asarray(labels)

            Y_, cache = forward_propagation(X, params)
            loss = compute_loss(Y_, Y)
            grads = backward_propagation(params, cache, X, Y)
            params = update_params(params, grads, learning_rate)

            total_loss += loss
            total_correct += np.sum(np.argmax(Y_, axis=0) == labels_gpu)
            total_samples += labels_gpu.shape[0]

            if i % 100 == 99:
                print('Epoch {}, iteration {}, loss : {}, accuracy : {}'.format(epoch+1, i+1, total_loss / 100, total_correct/ total_samples))
                total_loss = 0


        train_error_rate = (1 - (total_correct / total_samples)) * 100
        train_error_list.append(train_error_rate)

        total_loss = 0
        total_correct = 0
        total_samples = 0
        for i, (images, labels) in enumerate(test_loader):
            X = images.view(-1, 28 * 28)
            X = np.asarray(X).T
            Y = np.eye(10)[labels]
            Y = np.asarray(Y).T

            labels_gpu = np.asarray(labels)

            Y_, cache = forward_propagation(X, params)
            loss = compute_loss(Y_, Y)

            total_loss += loss
            total_correct += np.sum(np.argmax(Y_, axis=0) == labels_gpu)
            total_samples += labels_gpu.shape[0]

        test_error_rate = (1 - (total_correct / total_samples)) * 100
        test_error_list.append(test_error_rate)

        print('Test data - After epoch {}, loss : {}, accuracy : {}'.format(epoch+1, total_loss / len(test_loader), total_correct / total_samples))


    return params, train_error_list, test_error_list


def predict(X, params):
    Y_, cache = forward_propagation(X, params)
    predictions = np.argmax(Y_, axis=0)
    return predictions


def compute_accuracy(predictions, labels):
    return np.sum(predictions == labels) / len(labels)

def extract_features(params, loader):
    input_images = []
    first_hidden = []
    second_hidden = []
    third_hidden = []

    for images, labels in loader:
        X = images.view(-1, 28 * 28)
        X = np.asarray(X).T

        Y_, cache = forward_propagation(X, params)
        input_images.append(X.T)
        first_hidden.append(cache["H1"].T)
        second_hidden.append(cache["H2"].T)
        third_hidden.append(cache["H3"].T)

    return np.concatenate(input_images), np.concatenate(first_hidden), np.concatenate(second_hidden), np.concatenate(third_hidden)


def plot_tsne(features, title):
    tsne = TSNE(n_components=2, random_state=0)
    reduced_features = tsne.fit_transform(features)

    plt.figure(figsize=(6, 6))
    plt.scatter(reduced_features[:, 0], reduced_features[:, 1], marker='.', s=1)
    plt.title(title)
    plt.show()


In [ ]:
if __name__=="__main__":
    batch_size = 64

    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

    trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
    testset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=False, transform=transform)

    train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True)

    bp_params = initialize_bp_params()
    dfa_params = initialize_dfa_params()




In [ ]:
    print("Training with back propagation...")
    bp_params, bp_train_errors, bp_test_errors = train(backward_propagation, bp_params, train_loader, test_loader, learning_rate=0.01, num_epochs=10)

In [ ]:
    print("Training with direct feedback alignment...")
    dfa_params, dfa_train_errors, dfa_test_errors = train(direct_feedback_alignment, dfa_params, train_loader, test_loader, learning_rate=0.01, num_epochs=10)

In [ ]:
    epochs = range(1, len(bp_train_errors) + 1)

    plt.figure(figsize=(12,6))
    plt.plot(epochs, [err.get() for err in bp_train_errors], label='BP train')
    plt.plot(epochs, [err.get() for err in bp_test_errors], label='BP test')
    plt.plot(epochs, [err.get() for err in dfa_train_errors], label='DFA train')
    plt.plot(epochs, [err.get() for err in dfa_test_errors], label='DFA test')
    plt.xlabel('Epoch')
    plt.ylabel('Error rate(%)')
    plt.legend()
    plt.show()

In [ ]:
    print("Extracting and plotting features for back propagation...")
    bp_input_images, bp_first_hidden, bp_second_hidden, bp_third_hidden = extract_features(bp_params, test_loader)
    plot_tsne(bp_input_images, 'Backpropagation: Input Images')
    plot_tsne(bp_first_hidden, 'Backpropagation: First Hidden Layer')
    plot_tsne(bp_second_hidden, 'Backpropagation: Second Hidden Layer')
    plot_tsne(bp_third_hidden, 'Backpropagation: Third Hidden Layer')

    print("Extracting and plotting features for direct feedback alignment...")
    dfa_input_images, dfa_first_hidden, dfa_second_hidden, dfa_third_hidden = extract_features(dfa_params, test_loader)
    plot_tsne(dfa_input_images, 'Direct Feedback Alignment: Input Images')
    plot_tsne(dfa_first_hidden, 'Direct Feedback Alignment: First Hidden Layer')
    plot_tsne(dfa_second_hidden, 'Direct Feedback Alignment: Second Hidden Layer')
    plot_tsne(dfa_third_hidden, 'Direct Feedback Alignment: Third Hidden Layer')

# 追実験2

In [1]:
import cupy as np
import torch
from torchvision import datasets, transforms
%matplotlib inline
import matplotlib.pyplot as plt
import copy

def softmax(x):
    # ベクトルxの各要素から最大値を引く（オーバーフロー防止のため）
    e_x = np.exp(x - np.max(x))
    # 要素ごとのexpの和で各要素を割り、確率として解釈できるようにする
    return e_x / e_x.sum(axis=0)

def sigmoid(x):
    return 1 / (1+ np.exp(-x))

def initialize_bp_params():
    np.random.seed(1)
    W1 = np.random.randn(50, 784) * 0.01
    b1 = np.zeros((50, 1))
    W2 = np.random.randn(50, 400) * 0.01
    b2 = np.zeros((50, 1))
    W3 = np.random.randn(50, 50) * 0.01
    b3 = np.zeros((50, 1))
    W4 = np.random.randn(10, 50) * 0.01
    b4 = np.zeros((10, 1))

    params = {  "W1": W1,
                "b1": b1,
                "W2": W2,
                "b2": b2,
                "W3": W3,
                "b3": b3,
                "W4": W4,
                "b4": b4}

    return params

def initialize_dfa_params():
    np.random.seed(1)
    W1 = np.zeros((50, 784))
    b1 = np.zeros((50, 1))
    W2 = np.zeros((50, 50))
    b2 = np.zeros((50, 1))
    W3 = np.zeros((50, 50))
    b3 = np.zeros((50, 1))
    W4 = np.zeros((10, 50))
    b4 = np.zeros((10, 1))

    B1 = np.random.randn(50, 10) * 0.01
    B2 = np.random.randn(50, 10) * 0.01
    B3 = np.random.randn(50, 10) * 0.01

    params = {  "W1": W1,
                "b1": b1,
                "W2": W2,
                "b2": b2,
                "W3": W3,
                "b3": b3,
                "W4": W4,
                "b4": b4,
                "B1": B1,
                "B2": B2,
                "B3": B3}

    return params

def forward_propagation(X, params):
    W1 = params["W1"]
    b1 = params["b1"]
    W2 = params["W2"]
    b2 = params["b2"]
    W3 = params["W3"]
    b3 = params["b3"]
    W4 = params["W4"]
    b4 = params["b4"]

    A1 = np.dot(W1, X) + b1
    H1 = np.tanh(A1)
    A2 = np.dot(W2, H1) + b2
    H2 = np.tanh(A2)
    A3 = np.dot(W3, H2) + b3
    H3 = np.tanh(A3)
    AY = np.dot(W4, H3) + b4
    # Y_ = softmax(AY)
    Y_ = sigmoid(AY)

    cache = {"A1": A1,
            "H1": H1,
            "A2": A2,
            "H2": H2,
            "A3": A3,
            "H3": H3,
            "AY": AY,
            "Y_": Y_}

    return Y_, cache

def backward_propagation(params, cache, X, Y):
    # batch size
    N = X.shape[1]

    W1 = params["W1"]
    W2 = params["W2"]
    W3 = params["W3"]
    W4 = params["W4"]

    H1 = cache["H1"]
    H2 = cache["H2"]
    H3 = cache["H3"]
    Y_ = cache["Y_"]

    dY_ = (Y_ - Y)
    dW4 = (1 / N) * np.dot(dY_, H3.T)
    db4 = (1 / N) * np.sum(dY_, axis=1, keepdims=True)

    dH3 = np.dot(W4.T, dY_)
    dA3 = dH3 * (1 - np.power(H3, 2))
    dW3 = (1 / N) * np.dot(dA3, H2.T)
    db3 = (1 / N) * np.sum(dA3, axis=1, keepdims=True)

    dH2 = np.dot(W3.T, dA3)
    dA2 = dH2 * (1 - np.power(H2, 2))
    dW2 = (1 / N) * np.dot(dA2, H1.T)
    db2 = (1 / N) * np.sum(dA2, axis=1, keepdims=True)

    dH1 = np.dot(W2.T, dA2)
    dA1 = dH1 * (1 - np.power(H1, 2))
    dW1 = (1 / N) * np.dot(dA1, X.T)
    db1 = (1 / N) * np.sum(dA1, axis=1, keepdims=True)

    gradients = {"dW1": dW1,
                "db1": db1,
                "dW2": dW2,
                "db2": db2,
                "dW3": dW3,
                "db3": db3,
                "dW4": dW4,
                "db4": db4}

    return gradients

def direct_feedback_alignment(params, cache, X, Y):
    # batch size
    N = X.shape[1]

    B1 = params["B1"]
    B2 = params["B2"]
    B3 = params["B3"]

    H1 = cache["H1"]
    H2 = cache["H2"]
    H3 = cache["H3"]
    Y_ = cache["Y_"]

    dY_ = Y_ - Y
    dW4 = (1 / N) * np.dot(dY_, H3.T)
    db4 = (1 / N) * np.sum(dY_, axis=1, keepdims=True)

    dH3 = np.dot(B3, dY_)
    dA3 = dH3 * (1 - np.power(H3, 2))
    dW3 = (1 / N) * np.dot(dA3, H2.T)
    db3 = (1 / N) * np.sum(dA3, axis=1, keepdims=True)

    dH2 = np.dot(B2, dY_)
    dA2 = dH2 * (1 - np.power(H2, 2))
    dW2 = (1 / N) * np.dot(dA2, H1.T)
    db2 = (1 / N) * np.sum(dA2, axis=1, keepdims=True)

    dH1 = np.dot(B1, dY_)
    dA1 = dH1 * (1 - np.power(H1, 2))
    dW1 = (1 / N) * np.dot(dA1, X.T)
    db1 = (1 / N) * np.sum(dA1, axis=1, keepdims=True)

    gradients = {"dW1": dW1,
                "db1": db1,
                "dW2": dW2,
                "db2": db2,
                "dW3": dW3,
                "db3": db3,
                "dW4": dW4,
                "db4": db4}

    return gradients


def update_params(params, grads, learning_rate=0.01, is_weights_fixed=False):
    if not is_weights_fixed:
        params["W1"] -= learning_rate * grads["dW1"]
        params["b1"] -= learning_rate * grads["db1"]
    params["W2"] -= learning_rate * grads["dW2"]
    params["b2"] -= learning_rate * grads["db2"]
    params["W3"] -= learning_rate * grads["dW3"]
    params["b3"] -= learning_rate * grads["db3"]
    params["W4"] -= learning_rate * grads["dW4"]
    params["b4"] -= learning_rate * grads["db4"]
    return params


def compute_loss(Y_, Y):

    N = Y.shape[1]
    # loss = (-1 / N) * np.sum(np.multiply(Y, np.log(Y_)))
    loss = (-1 / N) * np.sum(np.multiply(Y, np.log(Y_)) + np.multiply(1 - Y, np.log(1 - Y_)))
    loss = np.squeeze(loss)

    return loss


def train(backward_propagation, params, train_loader, test_loader, train_error_list, test_error_list, learning_rate=0.01, start_epoch=0, num_epochs=50, is_weights_fixed=False):
    for epoch in range(num_epochs):
        total_loss = 0
        total_correct = 0
        total_samples = 0
        for i, (images, labels) in enumerate(train_loader):
            X = images.view(-1, 28 * 28)
            X = np.asarray(X).T
            Y = np.eye(10)[labels]
            Y = np.asarray(Y).T

            labels_gpu = np.asarray(labels)

            Y_, cache = forward_propagation(X, params)
            loss = compute_loss(Y_, Y)
            grads = backward_propagation(params, cache, X, Y)
            params = update_params(params, grads, learning_rate, is_weights_fixed=is_weights_fixed)

            total_loss += loss
            total_correct += np.sum(np.argmax(Y_, axis=0) == labels_gpu)
            total_samples += labels_gpu.shape[0]

            if i % 100 == 99:
                print('Epoch {}, iteration {}, loss : {}, accuracy : {}'.format(start_epoch+epoch+1, i+1, total_loss / 100, total_correct/ total_samples))
                total_loss = 0


        train_error_rate = (1 - (total_correct / total_samples)) * 100
        train_error_list.append(train_error_rate)

        total_loss = 0
        total_correct = 0
        total_samples = 0
        for i, (images, labels) in enumerate(test_loader):
            X = images.view(-1, 28 * 28)
            X = np.asarray(X).T
            Y = np.eye(10)[labels]
            Y = np.asarray(Y).T

            labels_gpu = np.asarray(labels)

            Y_, cache = forward_propagation(X, params)
            loss = compute_loss(Y_, Y)

            total_loss += loss
            total_correct += np.sum(np.argmax(Y_, axis=0) == labels_gpu)
            total_samples += labels_gpu.shape[0]

        test_error_rate = (1 - (total_correct / total_samples)) * 100
        test_error_list.append(test_error_rate)

        print('Test data - After epoch {}, loss : {}, accuracy : {}'.format(start_epoch+epoch+1, total_loss / len(test_loader), total_correct / total_samples))

    return params, train_error_list, test_error_list


def experiment(params, train_loader, test_loader, learning_rate=0.01):
    bp_train_error_list = []
    bp_test_error_list = []
    dfa_train_error_list = []
    dfa_test_error_list = []

    params, bp_train_error_list, bp_test_error_list = train(backward_propagation,
                                                            params,
                                                            train_loader,
                                                            test_loader,
                                                            bp_train_error_list,
                                                            bp_test_error_list,
                                                            learning_rate=learning_rate,
                                                            num_epochs=50,
                                                            is_weights_fixed=True)
    bp_params = copy.deepcopy(params)
    dfa_params = copy.deepcopy(params)
    dfa_train_error_list = copy.deepcopy(bp_train_error_list)
    dfa_test_error_list = copy.deepcopy(bp_test_error_list)

    bp_params, bp_train_error_list, bp_test_error_list = train(backward_propagation,
                                                            bp_params,
                                                            train_loader,
                                                            test_loader,
                                                            bp_train_error_list,
                                                            bp_test_error_list,
                                                            learning_rate=learning_rate,
                                                            start_epoch=50,
                                                            num_epochs=50,
                                                            is_weights_fixed=False)

    dfa_params, dfa_train_error_list, dfa_test_error_list = train(direct_feedback_alignment,
                                                            dfa_params,
                                                            train_loader,
                                                            test_loader,
                                                            dfa_train_error_list,
                                                            dfa_test_error_list,
                                                            learning_rate=learning_rate,
                                                            start_epoch=50,
                                                            num_epochs=50,
                                                            is_weights_fixed=False)



    return bp_params, dfa_params, bp_train_error_list, bp_test_error_list, dfa_train_error_list, dfa_test_error_list


def predict(X, params):
    Y_, cache = forward_propagation(X, params)
    predictions = np.argmax(Y_, axis=0)
    return predictions


def compute_accuracy(predictions, labels):
    return np.sum(predictions == labels) / len(labels)

In [ ]:
if __name__=='__main__':
    batch_size = 64

    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

    trainset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=True, transform=transform)
    testset = datasets.MNIST('~/.pytorch/MNIST_data/', download=True, train=False, transform=transform)

    train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=True)

    params = initialize_bp_params()


In [ ]:
    bp_params, dfa_params, bp_train_error_list, bp_test_error_list, dfa_train_error_list, dfa_test_error_list = experiment(params, train_loader, test_loader, learning_rate=0.01)